In [1]:
import re
import os
import io
import glob
import random
from pathlib import Path
import spacy
import docx2txt
import PyPDF2
import csv
import pandas
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.corpus import wordnet
from itertools import islice
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')
os.chdir("C:/Users/Krish/Desktop/Review")
docDir = "C:/Users/Krish/Desktop/Review/Resumes/"
txtDir = "C:/Users/Krish/Desktop/Review/Text/"
newDir = "C:/Users/Krish/Desktop/Review/New/"
import mysql.connector
file_name_dict={}
file_mail_dict={}
file_ph_dict={}
file_exp_dict={}
file_skill_dict={}
result_dict = {}

In [2]:
#function to remove the words from the spacy's stop words vocabulary
print('Number of stop words: %d' % len(STOP_WORDS))

removeStop = [ 'around', 'as', 'at', 'been', 'between', 'during', 'eight', 'eleven', 'fifteen', 'fifty', 'five' , 'for',
              'forty', 'four', 'from', 'hundred', 'nine', 'since', 'six', 'sixty', 'ten', 'three', 'twelve', 'twenty', 'two', 
             'upon', 'of', 'to']
for stop in removeStop:
    STOP_WORDS.remove(stop)

Number of stop words: 305


In [3]:
#function to add the words to the spacy's stop words vocabulary
with open('Stop.txt','r') as f:
    customize_stop_words =  f.read().split(',')
#print(customize_stop_words)    
for word in customize_stop_words:
    STOP_WORDS.add(word)
for word in STOP_WORDS:    
    lexeme=nlp.vocab[word]
    lexeme.is_stop=True
print(len(STOP_WORDS))

777


In [4]:
#function which the file extensions and if it is docx or pdf it calls the functions to convert them into txt format
def convertMultiple(docDir, txtDir):
    file_list=[]
    for doc in os.listdir(docDir): 
        doc_len=len(doc.split("."))
        fileExtension = doc.split(".")[-1]
        dname=doc.split(".")[0]
        print(dname)
        if fileExtension == "docx":
            #function to convert a docx file to a txt file
            docFilename = docDir + str(doc) 
            text = docx2txt.process(docFilename) 
            textFilename = txtDir + dname + ".txt"
            textFile = open(textFilename, "w", encoding="UTF-8") 
            textFile.write(text)
            textFile.close()
            
        elif fileExtension == "pdf":
            docFilename = docDir + str(doc) 
            textFilename = txtDir + dname + ".txt"
            text = pdftotext(docFilename,textFilename) 
            
            
        elif fileExtension =="txt":
            docFilename = docDir + str(doc) 
            textFilename = txtDir + dname + ".txt"
            textFile = open(textFilename, "w", encoding="UTF-8") 
            textFile.write(text)
            textFile.close()

In [5]:
#function to convert a pdf file to a txt file 
def pdftotext(file,textFilename):
    content = ""
    p1 = open(file, "rb")
    pdf = PyPDF2.PdfFileReader(p1)
    num=pdf.getNumPages()
    for i in range(0, num):
        content += pdf.getPage(i).extractText() + "\n"
    content = " ".join(content.replace(u"\xa0", u" ").strip().split())
    new=""
    for i in content:
        new+=i
    textFile = open(textFilename, "w", encoding="UTF-8") 
    textFile.write(new)
    textFile.close()
    p1.close()

In [6]:
#function to find e-mail ids
def email(newDir):
    path = newDir + "*.txt"
    path = path.replace(os.sep, '/')
    text_files = glob.glob(path)
    files = set(text_files)
    files = list(files)
    for each in files:
        filename=each.split('\\')[-1]
        filename=filename.split('.')[0]
        #print(filename)                    
        with open(each, 'r', encoding='utf8', errors='ignore') as f:
            search = f.read()
        email = None           
        pattern = re.compile('[\w\.-]+@[\w\.-]+')
        matches = pattern.findall(search)
        email = matches
        email = set(email)
        email = list(email)
        new = "" 
  
      
        for x in email: 
            new += x+" , " 
        print(new)    
        file_mail_dict[filename]=new
        

In [7]:
#function to find phone numbers
def phone(file):
    path = newDir + "*.txt"
    path = path.replace(os.sep, '/')
    text_files = glob.glob(path)
    files = set(text_files)
    files = list(files)   
    for each in files:
        filename=each.split('\\')[-1]
        filename=filename.split('.')[0]
        with open(each, 'r', encoding='utf8', errors='ignore') as f:
            search = f.read()
        phone = None           
        pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
        matches = pattern.findall(search)
        matches = [re.sub(r'[,.]', '', el) for el in matches if len(re.sub(r'[()\-.,\s+]', '', el))>6]
        matches = [re.sub(r'\D$', '', el).strip() for el in matches]
        matches = [el for el in matches if len(re.sub(r'\D','',el)) <= 15]
        for el in list(matches):
            if len(el.split('-')) > 3: continue
            for x in el.split("-"):
                try:
                    if x.strip()[-4:].isdigit():
                        if int(x.strip()[-4:]) in range(1900, 2100):
                            matches.remove(el)
                except: pass                  
        phone = matches
        phone = set(phone)
        phone = list(phone)
        new = "" 
  
      
        for x in phone: 
            new += x+" , " 
        print(new) 
        file_ph_dict[filename]=new



In [8]:
#function to find the skillsets
def skillsets():
    
    from gensim.models import Word2Vec
    model = Word2Vec.load('java_model.bin')
    vector=model.wv
    path = newDir + "*.txt"
    path = path.replace(os.sep, '/')
    #print(path)
    text_files = glob.glob(path)
    files = set(text_files)
    files = list(files) 
    #print(files)
    for each in files:
        filename=each.split('\\')[-1]
        filename=filename.split('.')[0]
        extension=each.split('.')[-1]
        if extension == 'txt':
            
        
            skillsets=set() 
            with open(each, 'r', encoding='utf8', errors='ignore') as f:
                search = f.read().lower().split()   
                    
                for i in search:
                    #compares each word in the file with java
                    if i in vector.vocab and i.isdigit()==False:                    
                        res=vector.similarity('java',i)
    
                    #stores only those values which are greater than 94%
                        if res>0.94 :
                            skillsets.add(i)
        new=""        
        
        skillsets=list(skillsets)
        for x in skillsets: 
            new += x+" , " 
        new=tokenize1(new)    
        print(new)
        file_skill_dict[filename]=new


        

In [9]:
#function to remove phone numbers for finding the name and experience
def remove_phone(file):
    with open(file, 'r', encoding='utf8', errors='ignore') as f:
        search = f.read()
    try:
    
        phone = None           
        pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
        matches = pattern.findall(search)
        matches = [re.sub(r'[,.]', '', el) for el in matches if len(re.sub(r'[()\-.,\s+]', '', el))>6]
        matches = [re.sub(r'\D$', '', el).strip() for el in matches]
        matches = [el for el in matches if len(re.sub(r'\D','',el)) <= 15]
        for el in list(matches):
            if len(el.split('-')) > 3: continue
            for x in el.split("-"):
                try:
                    if x.strip()[-4:].isdigit():
                        if int(x.strip()[-4:]) in range(1900, 2100):
                            matches.remove(el)
                except: pass                  
        phone = matches
        phone = set(phone)
        phone = list(phone)
        search = ""
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
                search = f.read()
                for i in phone:
                    search = search.replace(i," ")
        with open(file, "w",encoding='utf8', errors='ignore') as f:
            f.write(search)
    except:
        pass

In [10]:
#function to remove e-mails  for finding the name and experience
def remove_email(file):
    with open(file, 'r', encoding='utf8', errors='ignore') as f:
        search = f.read()
    try:    
        email = None           
        pattern = re.compile('[\w\.-]+@[\w\.-]+')
        matches = pattern.findall(search)
        email = matches
        email = set(email)
        email = list(email)
        search = ""
        with open(file, 'r', encoding='utf8', errors='ignore') as f:
                search = f.read()
                for i in email:
                    search = search.replace(i," ")
        with open(file, "w",encoding='utf8', errors='ignore') as f:
            f.write(search)
    except:
        pass

In [11]:
#function to find the name
def name(file):
    filename=file.split('\\')[-1]
    filename=filename.split('.')[0]
    with open(file, 'r', encoding='utf8', errors='ignore') as f:
        search = f.read().upper()
           
    Sentences = nltk.sent_tokenize(search)
    Tokens = []
    for Sent in Sentences:
        Tokens.append(nltk.word_tokenize(Sent)) 
    Words_List = [nltk.pos_tag(Token) for Token in Tokens]

    Nouns_List = []

    for List in Words_List:
        for Word in List:
            if re.match('[NN.*]', Word[1]):
                Nouns_List.append(Word[0])

    Names = []
    for Nouns in Nouns_List:
        if not wordnet.synsets(Nouns):
                Names.append(Nouns)
    
    
    Name = ' '.join(Names[:1])
    print("Name : ", Name)
    file_name_dict[filename]=Name


In [12]:
#function to find the experience
def exp(file):

    filename=file.split('\\')[-1]
    filename=filename.split('.')[0]
    with open(file, 'r', encoding='utf8', errors='ignore') as f:
        search = f.read()
    try: 
    #print(search)
        new_left1 = []
        new_left2 = []
        left = 0
        
        left2 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,7}experience(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,7}", search)
        if(left2 != None):
            left = left2.group()
        left4 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,2}years(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,2}", search)
        if(left4 != None):
            left = left4.group()
        left5 = re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,2}year(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,2}", search)
        if(left5 != None):
            left = left5.group()    
    
        if(left == 0):
            print("Experience : ", 0)
            return
    #print(left)
        left1 = re.findall('[0-9]{1,2}',left)
        left1_int = list(map(int, left1))
        #print(left1_int)
        for a in left1:
            for b in left1_int:
                if len(a) <= 2 and b < 30:
                    new_left1.append(a)
        left1 = ''.join(new_left1[0])
        left2 = re.findall('[0-9]{1,2}.[0-9]{1,2}',left)
    
        exp = []
        if not left2:
            exp.append(left1)
            exp = ''.join(left1)
        else:
            exp.append(left2)
            exp = ''.join(left2)
    except:
        exp=0
    #print(left1)
    #print(left2)
    #print(exp)
    print("Experience : ", exp)
    file_exp_dict[filename]=exp

In [13]:
#function to call the functions which finds experience and name
def name_and_exp_call():
    path = newDir + "*.txt"
    path = path.replace(os.sep, '/')
    text_files = glob.glob(path)
    files = set(text_files)
    files = list(files)
    #print(files)
    for file in files:
        #file = "samiulla-h.txt"
        remove_phone(file)
        remove_email(file)
        name(file)
        exp(file)
        print("\n")

In [14]:
#function to store the extracted info in a csv
def store_csv():
    with open("resume.csv",'w',encoding='utf8', errors='ignore') as fd:
        writer=csv.writer(fd)
        writer.writerow(["ID","Name","Experience","Skillset","Phone no","EmailID"])
        for key in file_name_dict.keys():
            writer.writerow([key,file_name_dict[key],file_exp_dict.get(key,0),file_skill_dict[key],file_ph_dict[key],file_mail_dict[key]])
    

In [15]:
#function to calculate score and store the final result in a csv
def score():
    from gensim.models import Word2Vec
    import csv
    import math
#loads the word2vec model created
    model = Word2Vec.load('java_model.bin')
    vector=model.wv

    count_scr=0
    score=0
    maximum=0

    result_dict = {}
    #open the resume csv which has the job seeker's info like name,experience,etc
    with open("resume.csv",'r') as resume_csv_data:
        csv_data = csv.DictReader(resume_csv_data)
        for resume_row in csv_data:
            ID = resume_row["ID"]
            experience = resume_row["Experience"]
            skills= tokenize1(resume_row["Skillset"]).split(",")
            #open the resume csv which has the companies info like job description,experience required,etc
            job_csv = csv.DictReader(open('job_description11.csv', mode='r'))
            for i,job_row in enumerate(job_csv):
                job_experience_required=job_row["experience"]
                job_description=job_row["description"]
                job_ID = job_row["company"]
                job_description=tokenize1(job_description).split(',')
                for skill in skills:                    
                    for des in job_description:
                        
                            
                        if skill in vector.vocab and des in vector.vocab:    
                            similarity=vector.similarity(skill,des)
                            if similarity > maximum:
                                    maximum=similarity
                            if similarity >=0.9999999999:    
                                    count_scr=count_scr+1
                    score=score+maximum
#score is calculated by adding the maximum similarity skill with the experience of the job seeker and if any skill is exactly 
#matched it is multiplied by 0.2 and added to the final score
                score=score+float(experience)+(count_scr*0.2)    
                print("ID:{} Job{} Score: {}".format(ID,job_ID,score))
                if result_dict.get(job_ID,0)==0:
                    result_dict[job_ID]=[[ID,score]]
                else:
                    result_dict[job_ID].append([ID,score])
                score=0
                count_scr=0
                maximum=0
    print(result_dict)

    for each in result_dict.keys():
        result_dict[each].sort(key=lambda x: x[1],reverse=True)
        result_dict[each]=result_dict[each][:10]

#storing the score info in a new csv - output_data    
    import pandas as pd
    #First update

    
    df = pd.read_csv("output_data.csv")
    if df.empty==True:
        data=result_dict

        with open("output_data.csv",'w') as fp:
            writer=csv.writer(fp)
            writer.writerow(["Company","EmployeeID/Score"])
            for each in data.keys():
                data[each].sort(key=lambda x : x[1],reverse=True)
                writer.writerow([each,data[each][:3]])

    #Dynamic update

    #data=result_dict
    else:
        data=result_dict
        n_dict={}

        with open("output_data.csv",'r') as fp:
            reader=csv.reader(fp)
            for i,row in enumerate(reader):
                if i%2==0 and i>0:
                    dat=list(row)
                    lst=eval(dat[1])+data[dat[0]]
                    n_dict[dat[0]]=lst

        data=n_dict

        with open("output_data.csv",'w') as fp:
            writer=csv.writer(fp)
            writer.writerow(["Company","EmployeeID/Score"])
            for each in data.keys():
                data[each].sort(key=lambda x : x[1],reverse=True)
                writer.writerow([each,data[each][:3]])


In [16]:
#function to pre-process the data
def tokenize(string, newFilename):

    doc = nlp(string)
    str1 = '\n'
    for word in (doc.sents): #used for sentence tokenization
        sentence = nlp(word.text) #used for text tokenization

        for token in sentence:
            #checks whether the word is a stop word , pronoun or a punctuation
            if token.is_stop!=True and token.is_punct != True and token.pos_ != 'PRON':  
                
                str1 = str1 + str(token.lemma_) #stores the lemmatized word in the variable str1
                with open(newFilename, "a+", encoding = "UTF-8") as f:

                    f.write(str1)
                    str1 = ' '
        str1 = '\n'


In [17]:
#function to refine the skillsets
def tokenize1(string):
    doc = nlp(string)
    str1 = ' '
    for word in (doc.sents):
        sentence = nlp(word.text)

        for token in sentence:
            if token.is_stop!=True and token.is_punct != True and token.pos_ != 'PRON':
                #print(token)
                
                
                str1 = str1 + str(token.lemma_)+","
              
    return str1
            

In [18]:
#function to check whether the file is in txt format
def parse(txtDir):
    path = txtDir + "*.txt"
    path = path.replace(os.sep, '/')
    text_files = glob.glob(path)
    files = set(text_files)
    files = list(files)   
    for each in files:
        each = each.replace(os.sep, '/')
        each = os.path.basename(each)
        
        
        tname, extension = each.split(".")
        newFilename = newDir + str(tname) + ".txt"
        if extension == "txt":
            textFilename = txtDir + str(each) 
        with open(textFilename, 'r', encoding="UTF-8") as f:
            string = f.read().lower()
            tokenize(string, newFilename)
            

In [19]:
#function to remove the resumes parsed for info
def remove_resumes():
    import shutil
    d='C:/Users/Krish/Desktop/Review/Resumes/' 
    filesToRemove = [os.path.join(d,f) for f in os.listdir(d)]
    for f in filesToRemove:
        os.remove(f)

In [20]:
#function to remove the files - the converted files and the preprocessed files
def remove_text():
    d='C:/Users/Krish/Desktop/Review/Text/' 
    filesToRemove = [os.path.join(d,f) for f in os.listdir(d)]
    for f in filesToRemove:
        os.remove(f)    
    d='C:/Users/Krish/Desktop/Review/New/' 
    filesToRemove = [os.path.join(d,f) for f in os.listdir(d)]
    for f in filesToRemove:
        os.remove(f)